# Data Preprocessing

## Importing Libraries
In this section, we are initializing our environment with three fundamental Python libraries that will facilitate data analysis, numerical operations, and file management. Here's a brief overview of each:

1. **pandas (`pd`)**: 
    - **Description**: pandas is a powerful library for data manipulation and analysis. It offers data structures like DataFrames and Series that simplify handling structured data.
    - **Usage**: Throughout this notebook, we'll utilize `pd` for tasks like data ingestion, cleaning, transformation, and aggregation.

2. **numpy (`np`)**:
    - **Description**: numpy is the core library for numerical computing in Python. It provides support for working with arrays (including mathematical functions to operate on these arrays), which is essential for various data science tasks.
    - **Usage**: We'll employ `np` whenever we need to perform mathematical and statistical operations, especially when dealing with arrays or matrices of data.

3. **os**:
    - **Description**: The os module provides a way of using operating system-dependent functionality, such as reading or writing to the file system.
    - **Usage**: In this notebook, `os` will be instrumental in tasks related to file management, such as directory creation, file path handling, and checking the presence of files.

By importing these libraries, we're equipping our notebook with the tools necessary to handle a wide range of data analysis and computational tasks.

In [20]:
import pandas as pd
import numpy as np
import os

## Setting Up Data Retrieval Infrastructure
This code segment establishes the foundation for managing and processing our financial data. It can be broken down into two main sections:
1. **Predefined Sheet Names**:
    - **Description**: We define a list called `sheet_names` that contains the names of sheets expected in the Excel files. Each of these sheets corresponds to a specific type of financial data.
    - **Usage**: This predefined list will be instrumental when reading and processing data from Excel files, ensuring that we access the correct sheets and maintain consistency in data handling.

2. **Retrieving File Paths for Data Processing:**
    - **Description**: The above code sets the path to the directory where all the Excel files (with financial data) are stored. It then constructs a list called `stocks`, containing the full paths to each of these files.
    - **Usage**: With this setup, we'll have easy access to all the Excel files in the `data` directory, facilitating bulk processing and analysis of the financial data contained in them.

In essence, this segment prepares our environment to efficiently load, access, and analyze the financial datasets saved in the Excel format.

In [21]:
sheet_names = [
    'Info',
    'Historical',
    'Income Statement',
    'Quarterly Income Statement',
    'Cashflow',
    'Institutional Holders',
    'Mutual Fund Holders',
    'Major Holders'
]

#riempire stocks di tutti i vari codici, fare la retrive di tutti i file e buttarli in df_stock per poi poter lavorare su tutti i dati
directory = "./data"
stocks = [os.path.join(directory, file) for file in os.listdir(directory)]
stocks = [stock.replace("\\", "/") for stock in stocks] # lambda per unificare i percorsi tra mac e windows

## Integration of Financial Data
Added columns:
- **Daily_Return**: Represents the daily return.
  
- **Target_1day**: Indicates whether the closing price the next day will be higher (1) or lower (0) than the current day's closing price.

- **Target_5days**: Specifies if the closing price 5 days into the future will be higher (1) or lower (0) than the closing price of the current day.

- **Target_30days**: Determines if the closing price 30 days into the future will be higher (1) or lower (0) compared to the current day's closing price.

- **Net income** (Income statement): This metric calculates a company's profit after all expenses and taxes have been deducted. It stands as the most critical measure for investors, showcasing the company's bottom line.

- **Diluted EPS** (Income statement): Measures the company's profit for each share of common stock, offering a clear picture of the company's profitability per share.

- **Total Revenue** (Income statement): Reflects the total sales a company achieves. It provides a snapshot of the company's top line growth.

- **Cost of revenue** (Income statement): Calculates the cost of the goods the company sells, an essential measure for gauging a company's profitability.

- **Operating revenue** (Income statement): Captures the non-production costs incurred by a company. It's vital to factor in operating expenses when assessing a company's profitability and cash flow potential.

- **Cash flow from continuing operating activities** (Cashflow): Represents the cash generated by a company from its primary business ventures.

- **Cash flow from continuing investing activities** (Cashflow): Indicates the cash a company earns from its investments, such as selling property or equipment.

- **Cash flow from continuing financing activities** (Cashflow): Reflects the cash a company secures from its financing activities, like issuing debt or equity.

We've merged the "Income Statement" and "Cashflow" sheets into one Excel document. Note: As these sheets contain yearly or quarterly financial data, a common approach is to carry forward the last known value for each day until new data becomes available. For certain fiscal years, the value will be NaN as we don't possess the data.

The dataframes for each stock were truncated at 06/30/2020 and were also individually exported for potential future use. Lastly, we combined all stocks into a singular final dataset, identifying each using the "Ticker" feature we've added.

Some companies lacked complete financial data for our analysis. Therefore, by mutual agreement, we chose to still consider such data, filling in the missing features with neutral values (0). This enables us to include these data points and add value to the analysis of companies whose data is available.

## Feature Engineering
- **Moving Averages**: We calculate short and long-term moving averages for the closing price, which are standard in algorithmic trading. For instance, moving averages over 5, 10, 30, and 50 days.
  
- **RSI (Relative Strength Index)**: This is a momentum indicator that can help identify if a stock is in an "overbought" or "oversold" condition.

- **MACD (Moving Average Convergence Divergence)**: Another momentum indicator.

- **Bollinger Bands**: These are based on moving averages and can help determine if a price is relatively high or low.

- **Volatility**: We might compute volatility as the standard deviation of daily returns over a specific timeframe.

### Financial Data Integration and Feature Engineering

This code block is designed to process financial data for multiple stocks, adding derived features, and integrating various sheets from Excel files into a unified DataFrame structure. Here's a step-by-step explanation:

1. **Initialization**:
    - `counter` is set to 0, which will be used later in the code.
    - `df` is an empty DataFrame that will store the consolidated data from all processed stocks.

2. **Iterating through Stock Files**:
    The loop goes through each file path in the `stocks` list, representing different stocks' data.

3. **Loading Excel Data**:
    The code reads an Excel file corresponding to a specific stock into the `df_stock` DataFrame.

4. **Processing 'Historical' Data**:
    - The 'Historical' sheet is loaded.
    - The date column is renamed and set as the index.
    - The 'Daily_Return' column is calculated to represent the daily percentage change in the closing price.
    - Three target columns (`Target_1day`, `Target_5days`, and `Target_30days`) are added to indicate whether the closing price in the future (1, 5, or 30 days respectively) will be higher than the current day's closing price.
    - Rows with NaN values (resulting from the target creation process) are dropped.

5. **Processing 'Income Statement' Data**:
    - The 'Income Statement' sheet is loaded and transposed for easier integration.
    - Key financial metrics (like 'Net Income', 'Diluted EPS', etc.) are selected.
    - Missing columns for the selected metrics are created and filled with NaN values.
    - The data is then merged with the historical stock data.

6. **Processing 'Cashflow' Data**:
    - Similar to the 'Income Statement' data, the 'Cashflow' sheet is loaded, transposed, and key metrics are selected.
    - The cashflow data is then merged with the existing DataFrame.

7. **Feature Engineering**:
    The following technical indicators and financial metrics are calculated and added to the DataFrame:
    - **Moving Averages**: Calculated for the closing prices over 5, 10, 30, and 50 days.
    - **RSI (Relative Strength Index)**: A momentum indicator.
    - **MACD**: Another momentum indicator with a corresponding signal line.
    - **Bollinger Bands**: Used to determine relative high or low prices.
    - **Volatility**: Calculated as the standard deviation of daily returns over a specified window.

8. **Final Processing**:
    - A 'Ticker' column is added to the DataFrame to store the stock's identifier.
    - NaN values in various columns are filled with neutral values (0).
    - The data is truncated to only include entries after '2020-06-30'.
    - Each processed stock's data is saved in the 'Processed' directory.
    - The processed stock's data is then appended to the consolidated DataFrame `df`.

In essence, this code processes financial data for multiple stocks, derives meaningful metrics, integrates different data sheets, and prepares the data for further analysis or modeling tasks.

In [22]:
counter = 0
df = pd.DataFrame()
for file in stocks:
    if file.split("/")[2][:-5] != "GS":
        df_stock = pd.ExcelFile(file)
        
        # prevent false postive warnings, reference_ https://stackoverflow.com/questions/20625582/how-to-deal-with-settingwithcopywarning-in-pandas
        pd.options.mode.chained_assignment = None # default='warn'
        
        # Loading the 'Historical' data stock
        historical_data = df_stock.parse('Historical')
        
        # Renaming and setting the Date column
        historical_data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)
        historical_data['Date'] = pd.to_datetime(historical_data['Date'])
        historical_data.set_index('Date', inplace=True)
        # Calculate daily return
        historical_data['Daily_Return'] = historical_data['Close'].pct_change()
        
        # Create target variables for next day, next 5 days and next 30 days
        historical_data['Target_1day'] = (historical_data['Close'].shift(-1) > historical_data['Close']).astype(int)
        historical_data['Target_5days'] = (historical_data['Close'].shift(-5) > historical_data['Close']).astype(int)
        historical_data['Target_30days'] = (historical_data['Close'].shift(-30) > historical_data['Close']).astype(int)
        
        # Drop rows with NaN values (will be present due to the shifting for target creation)
        historical_data = historical_data.dropna()
        
        # Loading the 'Income Statement' data for XOM
        income_statement = df_stock.parse('Income Statement')
        
        # Transposing the data for easier integration
        income_statement = income_statement.set_index('Unnamed: 0').transpose()
        income_statement.index = pd.to_datetime(income_statement.index)
        
        
        # Selecting some of the key financial metrics (you can add or remove based on relevance)
        selected_metrics = [
            'Net Income',
            'Diluted EPS',
            'Total Revenue',
            'Normalized EBITDA',
            'Total Unusual Items',
            'Total Unusual Items Excluding Goodwill'
        ]
        
        # check if columns exist, in case create them
        for metric in selected_metrics:
            if metric not in income_statement.columns:
                income_statement[metric] = np.nan
                
        
        income_statement = income_statement[selected_metrics]
        
        # Merging the income statement data with the historical data
        merged_data = historical_data.join(income_statement, how='left')
        
        # Forward filling the NaN values
        merged_data[selected_metrics] = merged_data[selected_metrics].fillna(method='ffill')
        
        # Loading the 'Cashflow' data for XOM
        cashflow = df_stock.parse('Cashflow')
        
        # Transposing the data for easier integration
        cashflow = cashflow.set_index('Unnamed: 0').transpose()
        cashflow.index = pd.to_datetime(cashflow.index)
        
        # Selecting some of the key cashflow metrics (you can add or remove based on relevance)
        selected_cashflow_metrics = [
            'Operating Cash Flow',
            'Capital Expenditure',
            'Free Cash Flow',
            'Cash Flow From Continuing Operating Activities',
            'Cash Flow From Continuing Investing Activities',
            'Cash Flow From Continuing Financing Activities'
        ]
        
        for metric in selected_cashflow_metrics:
            if metric not in cashflow.columns:
                cashflow[metric] = np.nan
        
        cashflow = cashflow[selected_cashflow_metrics]
        
        # Merging the cashflow data with the existing dataframe
        merged_data = merged_data.join(cashflow, how='left', rsuffix='_cashflow')
        
        # Forward filling the NaN values
        merged_data[selected_cashflow_metrics] = merged_data[selected_cashflow_metrics].fillna(method='ffill')
    
        # Moving Averages
        merged_data['MA_5'] = merged_data['Close'].rolling(window=5).mean()
        merged_data['MA_10'] = merged_data['Close'].rolling(window=10).mean()
        merged_data['MA_30'] = merged_data['Close'].rolling(window=30).mean()
        merged_data['MA_50'] = merged_data['Close'].rolling(window=50).mean()
        
        # RSI
        delta = merged_data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=14).mean()
        avg_loss = loss.rolling(window=14).mean()
        rs = avg_gain / avg_loss
        merged_data['RSI'] = 100 - (100 / (1 + rs))
        
        # MACD
        merged_data['MACD'] = merged_data['Close'].ewm(span=12, adjust=False).mean() - merged_data['Close'].ewm(span=26, adjust=False).mean()
        merged_data['Signal_Line'] = merged_data['MACD'].ewm(span=9, adjust=False).mean()
        
        # Bollinger Bands
        merged_data['Bollinger_Mid_Band'] = merged_data['Close'].rolling(window=20).mean()
        merged_data['Bollinger_Upper_Band']  = merged_data['Bollinger_Mid_Band'] + 1.96*merged_data['Close'].rolling(window=20).std()
        merged_data['Bollinger_Lower_Band']  = merged_data['Bollinger_Mid_Band'] - 1.96*merged_data['Close'].rolling(window=20).std()
        
        # Volatility
        merged_data['Volatility'] = merged_data['Daily_Return'].rolling(window=5).std()
        
        to_drop_na = ['MA_5', 'MA_10', 'MA_30', 'MA_50', 'RSI', 'Volatility', 'Cash Flow From Continuing Operating Activities', 'Cash Flow From Continuing Investing Activities', 'Cash Flow From Continuing Financing Activities', 'Net Income', 'Diluted EPS', 'Total Revenue','Normalized EBITDA', 'Total Unusual Items', 'Total Unusual Items Excluding Goodwill', 'Operating Cash Flow', 'Capital Expenditure','Free Cash Flow',]
    
        if 'Ticker' not in merged_data.columns:
                merged_data['Ticker'] = file.split("/")[2].replace(".", "")[:-4]
            
        # Display the updated dataframe with integrated cashflow metrics
        merged_data.iloc[counter : counter + len(merged_data), merged_data.columns.get_loc("Ticker")] = file.split("/")[2].replace(".", "")[:-4]
            
        counter = len(merged_data)
        
        for column in to_drop_na:
            merged_data[column] = merged_data[column].fillna(0)
    
        merged_data = merged_data[merged_data.index >= '2020-06-30']
        #indices_to_drop = merged_data.index[merged_data.isna().sum(axis=1) > 3].tolist()
        
        #merged_data.drop(indices_to_drop, inplace=True)
        
        # Export in Excel company data
        if not os.path.exists('./Processed'):
            os.makedirs('./Processed')
        with pd.ExcelWriter(f'./Processed/{file.split("/")[2][:-5]}.xlsx', mode = "w", engine = "openpyxl") as writer:
            merged_data.to_excel(writer, sheet_name="Sheet1")
        # Append to one single dataframe
        df = pd.concat([df, merged_data])

## Export del dataset finale

In [23]:
output_filepath = "final_dataset.xlsx"
df.to_excel(output_filepath)

In [24]:
# TODO: Rifinire il markdown del blocco di codice che fa tutto e gli altri due.
# TODO: Procedere al train del se il dataset viene considerato pronto.
# TODO: Replicare con le crypto.